<a href="https://colab.research.google.com/github/pedrofernandss/CIS_4Periodo/blob/main/CIS_5Periodo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dados = pd.read_csv('/content/drive/MyDrive/CIS/CIS_4Periodo/Musical_instruments_reviews.csv')

In [ ]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10261 non-null  float64
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 721.6+ KB


Há 27 células vazias na coluna "reviewerName" e 7 células vazias na coluna "reviewText". Tendo que se tratam de variáveis que são praticamente impossíveis de serem previstas, iremos remover as linhas.

In [ ]:
base_dados = base_dados.dropna()

In [ ]:
coluna_texto = base_dados.loc[:, ['reviewText']]
coluna_texto

,reviewText
0,"Not much to write about here, but it does exac..."
1,The product does exactly as it should and is q...
2,The primary job of this device is to block the...
3,Nice windscreen protects my MXL mic and preven...
4,This pop filter is great. It looks and perform...
...,...
10256,"Great, just as expected. Thank to all."
10257,I've been thinking about trying the Nanoweb st...
10258,I have tried coated strings in the past ( incl...
10259,"Well, MADE by Elixir and DEVELOPED with Taylor..."


In [ ]:
#Aplica a tokentização (separar cada palavra)
coluna_texto['reviewText'] = coluna_texto['reviewText'].apply(word_tokenize)
coluna_texto.head()

,reviewText
0,"[Not, much, to, write, about, here, ,, but, it..."
1,"[The, product, does, exactly, as, it, should, ..."
2,"[The, primary, job, of, this, device, is, to, ..."
3,"[Nice, windscreen, protects, my, MXL, mic, and..."
4,"[This, pop, filter, is, great, ., It, looks, a..."


In [ ]:
#Transformando todos os caracteres em minúsculos
coluna_texto['reviewText'] = coluna_texto['reviewText'].apply(lambda i: [palavra.lower() for palavra in i])
coluna_texto.head()

,reviewText
0,"[not, much, to, write, about, here, ,, but, it..."
1,"[the, product, does, exactly, as, it, should, ..."
2,"[the, primary, job, of, this, device, is, to, ..."
3,"[nice, windscreen, protects, my, mxl, mic, and..."
4,"[this, pop, filter, is, great, ., it, looks, a..."


In [ ]:
#Removendo pontuações
pontuações = ",.!;-_%$¨^`´~()"

coluna_texto['reviewText'] = coluna_texto['reviewText'].apply(lambda i: [palavra for palavra in i if palavra not in pontuações])
coluna_texto.head()

,reviewText
0,"[not, much, to, write, about, here, but, it, d..."
1,"[the, product, does, exactly, as, it, should, ..."
2,"[the, primary, job, of, this, device, is, to, ..."
3,"[nice, windscreen, protects, my, mxl, mic, and..."
4,"[this, pop, filter, is, great, it, looks, and,..."


In [ ]:
#Removendo preposições (stopwords)
lista_preposicoes = set(stopwords.words('english')) #pega as do inglês
coluna_texto['reviewText'] = coluna_texto['reviewText'].apply(lambda i: [palavra for palavra in i if palavra not in lista_preposicoes])
coluna_texto.head()

,reviewText
0,"[much, write, exactly, 's, supposed, filters, ..."
1,"[product, exactly, quite, affordable.i, realiz..."
2,"[primary, job, device, block, breath, would, o..."
3,"[nice, windscreen, protects, mxl, mic, prevent..."
4,"[pop, filter, great, looks, performs, like, st..."


In [ ]:
#Classificando gramaticalmente as palavras
coluna_texto['reviewText'] = coluna_texto['reviewText'].apply(nltk.tag.pos_tag)
coluna_texto.head()

,reviewText
0,"[(much, JJ), (write, WP), (exactly, RB), ('s, ..."
1,"[(product, NN), (exactly, RB), (quite, RB), (a..."
2,"[(primary, JJ), (job, NN), (device, NN), (bloc..."
3,"[(nice, JJ), (windscreen, JJ), (protects, NNS)..."
4,"[(pop, NN), (filter, NN), (great, JJ), (looks,..."


In [ ]:
#Cria função para #Reduzindo palavras à seus radicais (lemmatizer)
wnl = WordNetLemmatizer()
def lemmatize_words(word_tuples):
    return [wnl.lemmatize(word, pos=classe) if classe[0].isalpha() else word for word, classe in word_tuples]

#Aplica a função a coluna toda
wnl = WordNetLemmatizer()
coluna_texto['reviewText'] = coluna_texto['reviewText'].apply(lemmatize_words)

coluna_texto.head()

KeyError: ignored

In [ ]:
#Transforma as listas em string
coluna_texto['reviewText'] = [' '.join(l) for l in coluna_texto['reviewText']]

In [ ]:
vetorizar = CountVectorizer()
saco_palavras = vetorizar.fit_transform(coluna_texto['reviewText'])